In [8]:
from torchinfo import summary
from hparams import create_hparams
from model_utils import (
    load_classifier,
    prepare_dataloaders,
    parse_batch_classifier,
    load_final_model,
    load_model_checkpoint,
    parse_batch_generator,
    load_generator
)

In [9]:
hparams = create_hparams(local=True, log_output="print")
train_loader, _, val_loader, _ = prepare_dataloaders(hparams)
model = load_classifier(hparams, "cuda")

model_file = "/home/xzodia/dev/emo-gan/outputs/pretrain_discriminator_ESD/pretrain_7/"
model = load_final_model(model_file, model)

In [10]:
for batch in train_loader:
    x, _ = parse_batch_classifier(batch, hparams)
    break

In [11]:
summary(model, input_size=x.shape)

Layer (type:depth-idx)                   Output Shape              Param #
EmotionClassifier                        [8, 5]                    --
├─Conv2d: 1-1                            [8, 128, 800, 80]         5,888
├─LeakyReLU: 1-2                         [8, 128, 800, 80]         --
├─Dropout: 1-3                           [8, 128, 400, 20]         --
├─Conv2d: 1-4                            [8, 256, 400, 20]         491,776
├─LeakyReLU: 1-5                         [8, 256, 400, 20]         --
├─Dropout: 1-6                           [8, 256, 400, 20]         --
├─Conv2d: 1-7                            [8, 256, 400, 20]         983,296
├─LeakyReLU: 1-8                         [8, 256, 400, 20]         --
├─Dropout: 1-9                           [8, 256, 400, 20]         --
├─Conv2d: 1-10                           [8, 256, 400, 20]         983,296
├─LeakyReLU: 1-11                        [8, 256, 400, 20]         --
├─Dropout: 1-12                          [8, 256, 400, 20]         

In [12]:
model = load_generator(hparams, "cuda")
model_file = "/home/xzodia/dev/emo-gan/outputs/train_gan/train_6/generator/checkpoint_epoch_100.pth.tar"
model = load_model_checkpoint(model_file, model)

In [13]:
for batch in train_loader:
    x, _ = parse_batch_generator(batch)
    break

In [25]:
summary(model, input_data=(x,), depth=2)

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                [8, 80, 258]              --
├─Embedding: 1-1                         [8, 46, 512]              75,776
├─TextEncoder: 1-2                       [8, 46, 512]              --
│    └─ModuleList: 2-1                   --                        3,936,768
│    └─LSTM: 2-2                         [278, 512]                1,576,960
├─Linear: 1-3                            [8, 1, 128]               768
├─Decoder: 1-4                           [8, 80, 258]              --
│    └─Prenet: 2-3                       [130, 8, 256]             86,016
│    └─AttentionV2: 2-4                  --                        (recursive)
│    └─LSTMCell: 2-5                     [8, 512]                  2,887,680
│    └─AttentionV2: 2-6                  [8, 640]                  152,768
│    └─LSTMCell: 2-7                     [8, 512]                  3,411,968
│    └─LinearNorm: 2-8            